# Introduction to emodpy-hiv

`emodpy-hiv` is the new way for configuring, running, and analyzing EMOD-HIV simulations.  It replaces the previous DtkTools / HIV-Analayzers / run_scenarios.py.

One of the goals of this new tool is simplify the configuration of EMOD by allowing the user to do it via Python instead of JSON.  This gives the user the benefits that include:
- You don't have to worry about where parameters belong in deeply nested JSON.
- Configuration errors can be detected in the python code before the simulations are created.
- You can write a 'for-loop' when you only need to vary one value instead of copy and pasting large chunks of JSON.
- A Python IDE can provide tool-tips or allow you to navigate into functions so that you can see the definition of the arguments/parameters.

## The Packages of emodpy
The emodpy “framework” consists of a collection of packages.   Each package provides a collection of features that pertain to an aspect of configuring and running EMOD.

- **emod-api**
   - Defines base-level objects and how the input files of EMOD are written (or read).
- **emod-hiv** – (notice no “py”)
   - Contains a schema and Eradication binary
- **emodpy**
   - The common set of EMOD configuration features used by disease specific versions.  The user should not need to know much about this package.
- **emodpy-hiv/malaria**
   - The disease specific packages should be the main package that users use for configuring EMOD for the disease they are working on.  These packages are a “subclass” of emodpy.
- **idmtools**
   - Provides a “low-level” workflow framework.  It provides an abstract way to run EMOD on different platforms and provides a workflow that involves suites, experiments, and simulations.  By itself, it is model agnostic.  (This would be similar to a model-agnostic DtkTools.)

## Installation
- `mkdir emodpy_demo`
- `cd emodpy_demo`
- `python –m venv env`
- `env/Scripts/activate`
- `pip install --index-url https://packages.idmod.org/api/pypi/pypi-production/simple emodpy-hiv`

## Manifest
Manifest.py is used to store certain settings that would be common across scripts for running experiments or different for different people.

- Installed package includes a schema and executable
- Manifest allows you to specify other executables & schema.



In [ ]:
from IPython.display import Code

Code(filename='manifest.py', language='python')

## Eradication Binary/Executable - emod-hiv
The actual disease model for EMOD is an executable/binary file that is typically called Eradication(.exe).  The purpose of emodpy-hiv is to configure and run this binary.  Hence, you need a binary before you can do anything, so to make this easy an emodpy-hiv package requirement is that you also get the emod-hiv binary package (notice no "py" after the "emod").  This package contains an executable and an associated schema file that you can use with this version of emodpy-hiv.

Notice how the dtk.setup() method has an argument called `local_dir`.  The value of this argument will be the path/directory that the executable and schema files will get stored.  You want this to be the same directory you have specified in your manifest.py file for `schema_file` and `eradication_path`.

In [ ]:
import manifest
import emod_hiv.bootstrap as dtk

dtk.setup(local_dir=manifest.executables_dir)

## Platform
A Platform defines the type of cluster/computing platform that EMOD will be run on.  It provides an abstract way to run simulations on different hardware.

- `"SLURM_LOCAL"` - This value indicates that you are going to run your simulations on a SLURM-based HPC cluster.
- `job_directory` – The directory where your experiment & simulation directories will be created.
- https://docs.idmod.org/projects/idmtools/en/latest/platforms/slurm/index.html

In [ ]:
from idmtools.core.platform_factory import Platform

#platform = Platform( "SLURM_LOCAL", job_directory="experiments" )
platform = Platform( "Calculon", node_group="idm_abcd", priority="Normal" )

## EMODHIVTask
Configures the files for each simulation in an experiment.  It is found in the `emodpy-hiv` package.  The EMODHIVTask has been designed to specifically work with the emodpy-hiv country models, hence, do not use the emodpy.EMODTask directly.

- `eradication_path` – The path to the Eradication binary
- `schema_path` – The path to a processed schema.json file that comes from the Eradication binary 
- `param_custom_cb` – The function used to set the config parameters
- `campaign_builder` – The function that will build a campaign file for each simulation
- `demog_builder` – the function that will build a demographics file for each simulation
- `ep4_path` – The path to a directory with EMOD embedded python scripts

For this initial example, we are going to use the build methods that come with the Zambia country model.  Later, we will create our verions of the build methods so that we can change these defaults.

`emodpy_hiv.country_model` - This module contains code and data associated with different baseline country models.  These models provide you with default build methods so you can use them as-is or create your own build methods so that you can change the default behavior.

In [ ]:
# Will make the warnings off by default in 2.0
import emod_api.schema_to_class as s2c
s2c.show_warnings = False

import emodpy_hiv.emod_task as emod_task
import emodpy_hiv.country_model as cm

zambia = cm.Zambia()
task = emod_task.EMODHIVTask.from_default(
    eradication_path = manifest.eradication_path,
    schema_path      = manifest.schema_file,
    param_custom_cb  = zambia.build_config,
    campaign_builder = zambia.build_campaign,
    demog_builder    = zambia.build_demographics,
    ep4_path         = None
)

## Singularity Image
We use a a Singularity Image as a container that EMOD will run within.  This container provides all of the prerequisites (i.e. installation requirements) required by EMOD.  On SLURM, you must define the path to your image file.  Notice that the we are defining the path in your manifest file.

In [ ]:
#task.set_sif( path_to_sif=manifest.sif_path, platform=platform )
task.set_sif( path_to_sif=manifest.sif_path )

## SimulationBuilder
One more key element that we need for our experiment is a SimulationBuilder.  We use it to build/create the collection of simulations to include in the experiment.  The builder uses the concept of "parameter sweeps" to determine which simulations to create.  This means that we can define sweeps of different parameters to create different combinations of parameters.  The builder will create one simulation for each combination.

For our first time creating a builder, we will do a simple sweep on Run_Number.  Run_Number is the random number seed so we should get a different statistical realization of our scenario.

The first thing we do is to define a function for changing the parameter.  config parameters are very easy to change, but as we will see in a later tutorial, the campaign and demographics require a little more work.  It is not much, but it is different.  Notice that the the function returns a "tag" name and its corresponding value.  The tags can be used to sort/query your simulations based on these tags.

Once we have this function, notice that we add it to the builder saying that it is a "sweep definition".  In this definition, we specify our function and the list of values we want for the parameter.  We list three values so we should end up with three simulation runs where all of the inputs are the same except for this Run_Number parameter.

In [ ]:
from idmtools.builders import SimulationBuilder

def sweep_run_number( simulation, value ):
    simulation.task.config.parameters.Run_Number = value
    return { "Run_Number": value }


builder = SimulationBuilder()
builder.add_sweep_definition( sweep_run_number, [1,2,3] )

## Experiment
In EMOD vocabulary, an experiment is a collection of simulations or relaizations.  Typically, a user decides that a collection of parameter sweeps belong together and defines an "experiement".

The first thing we do is to create an experiment using our `builder` and `task`.  In this collection of tutorials, we will be doing it "from a builder", but there are other ways to create experiments.

Once we have our experiment created, we are ready to run our simulations.  Our experiement object will take care of everything for us required to run the simulations on our platform.  It will do things such as:
- submit jobs on the platform/cluster
- manage the running of all the sims
- retry a simulation if it fails
- allow you to have a margarita while it is doing the work

Notice that we are setting the `wait_until_done` parameter to `True`.  This means that this python program will wait at this line of code until all of the simulations have completed.  Hence, the execution of that line may take several minutes.

In [ ]:
from idmtools.entities.experiment import Experiment

experiment = Experiment.from_builder( builder, task, name="Tutorial_1" )

experiment.run( wait_until_done=True, platform=platform )

## Results
For Tutorial #1, you need to check "experiments" directory that that we defined when we created the Platform.  You should see a folder in there for your first experiment.  In that folder, you should see three simulation folders.  In each of those folders, there should be an "output" folder and it should contain a file called "InsetChart.json".  If you have that, you are ready to start Tutorial #2.